<a href="https://colab.research.google.com/github/PedroAlmeida18/PGP3_TESTESIND/blob/main/Carregamento_de_dados_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import torch
from torch import nn , optim
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd

args = {
    'batch_size':20,
    'num_workers':4,
    'num_classes':10,
    'lr':1e-4,
    'weight_decay':5e-4,
    'num_epocas':30
}
if torch.cuda.is_available():
  args['device'] = torch.device('cuda')
else :
  args['device'] = torch.device('gpu')
print(args['device'])


cuda


#Dataset


In [35]:
#baixar o dataset, pegar dados sobre clima, tempo, feriados, horarios para prever quantas bicicletas serão alugadas
! wget https://archive.ics.uci.edu/ml/machine-learning-databases/00275/Bike-Sharing-Dataset.zip
! unzip Bike-Sharing-Dataset.zip


--2024-01-09 18:02:08--  https://archive.ics.uci.edu/ml/machine-learning-databases/00275/Bike-Sharing-Dataset.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘Bike-Sharing-Dataset.zip.2’

Bike-Sharing-Datase     [   <=>              ] 273.43K   415KB/s    in 0.7s    

2024-01-09 18:02:09 (415 KB/s) - ‘Bike-Sharing-Dataset.zip.2’ saved [279992]

Archive:  Bike-Sharing-Dataset.zip
replace Readme.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [36]:
!ls

Bike-Sharing-Dataset.zip    Bike-Sharing-Dataset.zip.2	bike_train.csv	hour.csv    sample_data
Bike-Sharing-Dataset.zip.1  bike_test.csv		day.csv		Readme.txt


#Vizualizar dados

In [37]:
df = pd.read_csv('hour.csv')
print(len(df))
df.head()

17379


,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


#TRATAMENTO DE DADOS:
Separação de ddaos em treino e teste

In [38]:
torch.manual_seed(1)
indices = torch.randperm(len(df)).tolist()

train_size = int(0.8*len(df))

df_train=df.iloc[indices[:train_size]]
df_test=df.iloc[indices[train_size:]]

print(len(df_train),len(df_test))
display(df_test.head())

df_train.to_csv('bike_train.csv',index=False)
df_test.to_csv('bike_test.csv',index=False)
!ls

13903 3476


,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
12663,12664,2012-06-16,2,1,6,20,0,6,0,2,0.66,0.6212,0.47,0.1940,123,229,352
1801,1802,2011-03-20,1,0,3,18,0,0,0,1,0.38,0.3939,0.40,0.3582,58,98,156
16567,16568,2012-11-28,4,1,11,1,0,3,1,2,0.26,0.2576,0.75,0.2239,0,12,12
8817,8818,2012-01-08,1,1,1,5,0,0,0,2,0.32,0.3333,0.49,0.1045,0,2,2
2608,2609,2011-04-23,2,0,4,14,0,6,0,1,0.58,0.5455,0.78,0.3582,182,209,391


Bike-Sharing-Dataset.zip    Bike-Sharing-Dataset.zip.2	bike_train.csv	hour.csv    sample_data
Bike-Sharing-Dataset.zip.1  bike_test.csv		day.csv		Readme.txt


#Classe Dataset


In [39]:
class Bike(Dataset):

   #Criação da rede neural
  def __init__(self,csv_path):
    self.dados=pd.read_csv(csv_path).to_numpy()


  def __getitem__(self, index):
    sample = self.dados[index][2:14]
    label = self.dados[index][-1]
    #conversão pra tensor
    sample = torch.from_numpy(sample.astype(np.float32))
    label = torch.tensor(float(label), dtype=torch.float32)



    return sample, label

  def __len__(self):
    return len(self.dados)




Visualização dos dados

In [40]:
train_set= Bike('bike_train.csv')
test_set= Bike('bike_test.csv')
dado , rotulo = train_set[0]
print(rotulo)
print(dado)

tensor(373.)
tensor([ 4.0000,  1.0000, 11.0000, 19.0000,  0.0000,  4.0000,  1.0000,  1.0000,
         0.3800,  0.3939,  0.2700,  0.3582])


#Dataloader

In [41]:
train_loader = DataLoader(train_set,
                          batch_size = args['batch_size'],
                          shuffle=True,
                          num_workers= args['num_workers'])

test_loader = DataLoader(test_set,
                          batch_size = args['batch_size'],
                          shuffle=True,
                         num_workers= args['num_workers'])

#Realizar a indexação com o uso de for
#Como fazer para indexar dentro de um for, com o uso de quebrar a tupla
for batch in train_loader:
  dado, rotulo = batch
  print(dado.size(), rotulo.size())
  break

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


torch.Size([20, 12]) torch.Size([20])


#MLP

In [51]:
class MLP(nn.Module):

  def __init__(self, input_size, hidden_size, out_size):
    super(MLP, self).__init__()

    self.features = nn.Sequential(
          nn.Linear(input_size, hidden_size),
          nn.ReLU(),
          nn.Linear(hidden_size, hidden_size),
          nn.ReLU(),
    )

    self.classifier = nn.Sequential(
        nn.Linear(hidden_size, out_size),
        nn.ReLU(),
    )

  def forward(self, X):

    hidden = self.features(X)
    output = self.classifier(hidden)

    return output

input_size  = train_set[0][0].size(0)
hidden_size = 128
out_size    = 1

net = MLP(input_size, hidden_size, out_size).to(args['device'])
print(net)

MLP(
  (features): Sequential(
    (0): Linear(in_features=12, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): ReLU()
  )
  (classifier): Sequential(
    (0): Linear(in_features=128, out_features=1, bias=True)
    (1): ReLU()
  )
)


#Loss e otimizador

In [43]:
criterion = nn.L1Loss().to(args['device'])
optimizer =optim.Adam(net.parameters(),lr=args['lr'],weight_decay=args['weight_decay'])

#Fluxo de Treinamento

In [44]:
def train(train_loader,net,epocas):
  net.train()

  epoca_loss=[]
  for batch in train_loader:
        dado, rotulo = batch

        #cast na GPU

        dado = dado.to(args['device'])
        rotulo = rotulo.to(args['device'])

        #Forward
        pred = net(dado)
        loss = criterion(pred, rotulo)
        epoca_loss.append(loss.cpu().data)

        #backward
        loss.backward()
        optimizer.step()
  epoca_loss = np.asarray(epoca_loss)
  print("Epoca%d, Loss:%.4f +\- %.4f" % (epocas,epoca_loss.mean(),epoca_loss.std()))




#Validação
Mudança na Rede para o modo eval, infroma as camadaas que não tem mais calculo de gradiente
evitar o calculo de gardiente a mais

In [45]:
def test(test_loader,net,epocas):

   net.eval()
   with torch.no_grad():

    epoca_loss=[]
    for batch in test_loader:
          dado, rotulo = batch

          #cast na GPU

          dado = dado.to(args['device'])
          rotulo = rotulo.to(args['device'])

          #Forward
          pred = net(dado)
          loss = criterion(pred, rotulo)
          epoca_loss.append(loss.cpu().data)


    epoca_loss = np.asarray(epoca_loss)
    print("Epoca%d, Loss:%.4f +\- %.4f" % (epocas,epoca_loss.mean(),epoca_loss.std()))

In [54]:
for epocas in range(args['num_epocas']):
  train(train_loader,net,epocas)
  test(test_loader,net,epocas)
  print("______________________________________")

Epoca0, Loss:189.5791 +\- 40.4783
Epoca0, Loss:188.5309 +\- 38.5747
______________________________________
Epoca1, Loss:189.6390 +\- 41.5310
Epoca1, Loss:188.5077 +\- 39.9484
______________________________________
Epoca2, Loss:189.5657 +\- 40.9688
Epoca2, Loss:188.5975 +\- 39.6763
______________________________________
Epoca3, Loss:189.7293 +\- 41.8651
Epoca3, Loss:188.6098 +\- 38.5101
______________________________________
Epoca4, Loss:189.7684 +\- 39.3677
Epoca4, Loss:188.5965 +\- 36.7504
______________________________________
Epoca5, Loss:189.6406 +\- 41.8770
Epoca5, Loss:188.5265 +\- 42.4933
______________________________________
Epoca6, Loss:189.7289 +\- 39.9220
Epoca6, Loss:188.5578 +\- 38.9878
______________________________________
Epoca7, Loss:189.5478 +\- 42.9580
Epoca7, Loss:188.5743 +\- 41.7955
______________________________________
Epoca8, Loss:189.5466 +\- 42.2493
Epoca8, Loss:188.5815 +\- 38.6474
______________________________________
Epoca9, Loss:189.7200 +\- 41.1380
Epo

#Gráfico